## Work that might not be included in the final notebook for exam project

## Missing values work -- waiting until access is granted to API

#### Find missing address (street name) from geolocation

In [ ]:
#import urllib2
import numpy as np
import re
import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap
#from mpl_toolkits.mplot3d import Axes3D
from plotly import tools
%matplotlib inline
from scipy import stats, linalg
import pandas as pd
from collections import Counter 
from operator import itemgetter
import geoplotlib as gpl
from geoplotlib.utils import BoundingBox
import calendar
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import pylab as pl
#from sklearn import neighbors, datasets
#from sklearn.neighbors import KNeighborsClassifier
import geopy
import json

# Plotting with plotly
import plotly 
from IPython.display import Image
# Henriettes plotly API key og brugernavn -- gemmer plots i skyen
plotly.tools.set_credentials_file(username='frksteenhoff2', api_key='duu8hsfRmuI5rF2EU8o5')
import plotly.plotly as py
import plotly.graph_objs as go
import externalFunctions as ex

# Plotting color variables
bgBorder  = 'rgba(255, 255, 255, 0)'
ticksAxes = 'rgb(107, 107, 107)'
years_of_interest = [2013, 2014, 2015, 2016]

## The actual work

In [ ]:
# Replce missing borough values with "unspecified"
traffic_data['BOROUGH']        = traffic_data['BOROUGH'].replace(np.NaN, 'Unspecified')
traffic_data['LOCATION']       = traffic_data['LOCATION'].replace(np.NaN, 'Unspecified')
traffic_data['ON STREET NAME'] = traffic_data['ON STREET NAME'].replace(np.NaN, 'Unspecified')

# Extract the unspecified values
missing_bor = traffic_data.loc[traffic_data['BOROUGH']        == 'Unspecified']
missing_loc = traffic_data.loc[traffic_data['LOCATION']       == 'Unspecified']
missing_st  = traffic_data.loc[traffic_data['ON STREET NAME'] == 'Unspecified']

In [ ]:
# Extract all missing addresses that has a geo location
mis_address_w_geo = missing_st.loc[missing_st['LOCATION'] != 'Unspecified']
geo = list(mis_address_w_geo['LOCATION'].unique())
print "Number of unique missing addresses", len(geo)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from random import randint
from time import sleep

# Initialize list of street names
mis_adr = []

# Extract lon/lat from location
for i in range(0, len(geo)):
    lon = geo[0].split(",")[0].strip("(")
    lat = geo[1].split(",")[1].strip(")").strip(" ")
    location = geolocator.reverse(lon + "," + lat)
    mis_adr.append((location.address).split(",")[2])
    sleep(randint(10,100))
    

In [ ]:
mis_geo_w_addr = missing_loc.loc[missing_loc['ON STREET NAME'] != 'Unspecified']
print len(mis_geo_w_addr)
adr = list(mis_geo_w_addr['ON STREET NAME'])

In [ ]:
# Find out what can be done by using geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()

mis_boroughs = []
# Find boroughs for as many addresses as possible 
for i in range(0, len(adr)):
    location = geolocator.geocode(adr[i] +  "New York City")
    
    if location != None:
        # Show full address
        #print("\n" + location.address)

        # Extract borough from address
        #print(location.address.split(",")[2:3].strip(" "))

        # Extract basic address from address
        #print(location.address.split(",")[0].strip(" "))
        #print((location.latitude, location.longitude))
        
        mis_boroughs.append(str(location.address.split(",")[2:3]).strip(" "))
    else:
        #print "\nDamn it! Address at %d not found" % i
        mis_boroughs.append(np.NaN)

## Extracting traffic density information
This data is fetched from a NYC [Traffic Volume Report](https://l.facebook.com/l.php?u=https%3A%2F%2Fwww.dot.ny.gov%2Fdivisions%2Fengineering%2Ftechnical-services%2Fhds-respository%2FNYSDOT_2015TrafficVolumeReport-LocalRoads.pdf&h=ATOc31eRGO0FZDuZgAKOINQPGALSqJ9PT7XQIf8bpcYoDwuXV9agqxqTWALg0SpIRIQS9wZJaTo28nzTB3_X-P782waRplcXmtqZ1inoSjehe4hdeGV91VjxwLFRswv-tO-6fE13)

We have extracted only the information we need for the 5 boroughs we are working on.

In [ ]:
traffic_density = pd.DataFrame()
density_merge = pd.DataFrame()

# Reading in all sheets from excel sheet to create one large dataframe of denisty information
for i in range(1,112):
    traffic_density = pd.read_excel('NYC_traffic_density.xlsx', sheetname="Table "+str(i), skiprows=4, header=1)
    density_merge = density_merge.append(traffic_density)

# Find columns starting with "Unnamed"
filter_col = [col for col in list(density_merge) if not col.startswith('Unnamed')]
# Keep only columns without "Unnamed values"
density_merge = density_merge[filter_col]

# Save file to excel (only valid columns)
writer = pd.ExcelWriter('density.xlsx')
density_merge.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
# reading in the slightly edited file and selecting needed features
density = pd.read_excel('density_2.xlsx')
# Annual average daily traffic
needed_dens = density[['Borough','AADT', 'Beginning Description', 'End Description', 'Road Name']]
needed_dens[:10]

## Traffic info

**2016 - info**

``PASSENGER VEHICLE``
* Over the entire map. 
* Not as much in August/September

``NaN``
* Nearly none in January/February
* lots centered around Lower Manhattan..

``STATION WAGON``
* Wide spread
* High decrease from May to August
* Impact on all boroughs


``TAXI``
* Impact most in Manhattan/Brooklyn area
* HIGHLY centered around Times Square
* Mostly in fall/spring
* April/May/June/July close to no accidents

``SCOOTER``
* Almost no incidents -- not very exciting

``BUS``
* Only January to March?

``VAN``
* Not that interesting
* Few incidents from August to December